In [22]:
import chromadb
import polars as pl
from dotenv import load_dotenv
import os

load_dotenv()

articles = pl.read_csv('Articles.csv',  encoding='ISO-8859-1').with_row_index(offset = 1)

articles.head()

chroma_client = chromadb.Client()

articles = articles[:50]


In [23]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

model = HuggingFaceInferenceAPIEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    api_key= os.getenv('api_key')
)

list_articles = articles['Article'].to_list()

vectors =model.embed_documents(list_articles)
ids = [f"id{x}" for x in articles['index'].to_list()]


In [24]:
collection = chroma_client.get_or_create_collection(name='articles')

collection.add(
    documents=list_articles,
    ids=ids,
    embeddings=vectors,
)


Insert of existing embedding ID: id1
Insert of existing embedding ID: id2
Insert of existing embedding ID: id3
Insert of existing embedding ID: id4
Insert of existing embedding ID: id5
Insert of existing embedding ID: id6
Insert of existing embedding ID: id7
Insert of existing embedding ID: id8
Insert of existing embedding ID: id9
Insert of existing embedding ID: id10
Insert of existing embedding ID: id11
Insert of existing embedding ID: id12
Insert of existing embedding ID: id13
Insert of existing embedding ID: id14
Insert of existing embedding ID: id15
Insert of existing embedding ID: id16
Insert of existing embedding ID: id17
Insert of existing embedding ID: id18
Insert of existing embedding ID: id19
Insert of existing embedding ID: id20
Insert of existing embedding ID: id21
Insert of existing embedding ID: id22
Insert of existing embedding ID: id23
Insert of existing embedding ID: id24
Insert of existing embedding ID: id25
Insert of existing embedding ID: id26
Insert of existing em

In [25]:
query = "who is the Federal Finance Minister of Pakistan"

query_emb = model.embed_query(query)

context = collection.query(
    query_embeddings=query_emb,
    n_results=5
)

context

{'ids': [['id23', 'id27', 'id34', 'id44', 'id22']],
 'embeddings': None,
 'documents': [['ISLAMABAD: Pakistan\'s ongoing fuel shortage that has led to worsening power blackouts is weighing on its credit worthiness and hindering its ability to meet key reform targets laid out by the IMF, ratings agency Moody´s warned Monday. The country is currently in the grip of one of its worst power crises in years due to a shortfall in imported oil, with the situation exacerbated Sunday by an attack on a key powerline in restive Balochistan province. Moody´s said that increasing energy imports without addressing structural issues that create so-called circular debt "will further strain Pakistan\'s budget and balance of payments, a credit negative". "Fuel shortages also reflect the strained finances of state-owned distribution companies and the fuel importer, Pakistan State Oil corporation, and are a setback to the sector\'s progress on reforms made so far under Pakistan\'s financial support program

In [26]:
import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY")
)

# print(context)

context = str(context)
query = str(query)

chat_completion = client.chat.completions.create( 
    messages=[
        {
            "role": "system",
            "content": "You are an AI assistant. Use the following information provided by the user to answer questions accurately: \n\n" + context
        },
        {
            "role": "user",
            "content": "Based on the provided information, " + query
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)


Based on the provided information, the Federal Finance Minister of Pakistan is Ishaq Dar.
